# Eddy Detector

Term project for ESS 490/590

Grad: Erik Fredrickson

Undergrad: Ashika Capirala

*Some kind of project description can go here*

put a reminder here about the dependencies, and include an example line of how to load the environment file (should be included in our directory)

In [ ]:
# imports (WILL NEED TO BE UPDATED AS WE GO!)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from math import cos, sin, pi, sqrt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Import the Obspy modules that will be useful to download seismic data
import obspy
import obspy.clients.fdsn.client as fdsn
from obspy import UTCDateTime

### Import Data

Get bottom temperature and bottom pressure from IRIS

In [ ]:
# Pull pressure and temperature data from IRIS
network = 'XO'
staNames = ['LA21', 'LA34', 'LA33', 'LA23', 'LA25', 'LA22', 'LA28', 'LA39', 'LA32', 'LA30', 'LT07', 'LT06', \
           'LT13', 'LT03', 'LT11', 'LT04', 'LT01', 'LT20', 'LT14', 'LT16', 'LT10', 'LT12']
staCodes = 'LA21,LA34,LA33,LA23,LA25,LA22,LA28,LA39,LA32,LA30,LT07,LT06,LT13,LT03,LT11,LT04,LT01,LT20,LT14,LT16,LT10,LT12'
chaNames = ['HDH', 'HKO']
chaCodes='HDH,HKO'
Tstart = UTCDateTime(2018, 06, 01)
Tend = UTCDateTime(2019, 06, 20)

fdsn_client = fdsn.Client('IRIS')

# DO NOT RUN AS WRITTEN -- way too much data, so we'll need to make a loop to parse it by station and by day
Dtmp = fdsn_client.get_waveforms(network=network, station=staCodes, location='--', channel=chaCodes, starttime=Tstart, \
    endtime=Tend, attach_response=False)